In [1]:
import numpy as np
import torch

from scipy.optimize       import minimize
from libraries.graph      import graph_POSCAR_encoding, find_closest_key, composition_concentration_from_keys
from torch_geometric.data import Data
from pymatgen.core        import Structure

# Graph embedding

In [2]:
distance_threshold = 5
y = 1

# Whether to care about lattice vectors or not
is_molecule = False

structure = Structure.from_file('POSCAR')

nodes, edges, attributes = graph_POSCAR_encoding(structure,
                                                 distance_threshold=distance_threshold,
                                                 encoding_type='sphere-images')
temp = Data(x=nodes,
            edge_index=edges.t().contiguous(),
            edge_attr=attributes.flatten(),
            y=torch.tensor([y], dtype=torch.float)
           )
temp

Data(x=[72, 4], edge_index=[2, 601], edge_attr=[601], y=[1])

In [3]:
edges = temp.edge_index.detach().cpu().numpy().T
weights = temp.edge_attr.detach().cpu().numpy()

In [4]:
coordinates = []
for s in structure:
    coordinates.append(s.coords)
coordinates = np.array(coordinates)

In [5]:
lattice_vectors = structure.lattice.matrix
lattice_vectors

array([[ 1.26111574e+01,  1.12000000e-06,  4.49000000e-05],
       [ 1.73000000e-06,  1.71582865e+01, -2.53000000e-06],
       [ 3.68000000e-05, -1.57000000e-06,  1.02820259e+01]])

In [ ]:
mw = np.max(weights)
weights /= mw

## Molecules

In [6]:
if is_molecule:
    # Initial guess for the positions
    #initial_positions = np.random.rand(len(edges) * 3)  # Initialize all points at origin, 1D array
    initial_positions = coordinates.reshape(-1, 1).ravel()
    
    # Function to calculate the squared difference between distances and weights
    def objective(solution_attempt, edges, weights):
        positions = solution_attempt.reshape(-1, 3)  # Reshape to 2D array
        errors = 0
        for edge, weight in zip(edges, weights):
            p1 = positions[edge[0]]
            p2 = positions[edge[1]]
            distance = np.linalg.norm(p2 - p1)
            errors += np.power(distance - weight, 2)
        print(errors)
        return errors

## Crystals

In [7]:
if not is_molecule:
    # Initial guess for the lattice parameters
    lattice_vectors = np.array([[10, 0,   0],
                                [0,   10, 0],
                                [0,   0,   10]])
    
    # Initial guess for the positions
    initial_positions = np.random.rand(len(edges) * 3)*10  # Initialize all points at origin, 1D array
    #initial_positions = coordinates.reshape(-1, 1).ravel()
    #initial_positions = np.concatenate([lattice_vectors.ravel(), initial_positions])
    
    # Function to calculate the squared difference between distances and weights
    def objective(solution_attempt, edges, weights):
        solution_attempt = solution_attempt.reshape(-1, 3)  # Reshape to 2D array
        
        lattice_vectors = solution_attempt[:3]
        positions       = solution_attempt[3:]
        
        errors = 0
        for edge, weight in zip(edges, weights):
            p1 = positions[edge[0]]
            p2 = positions[edge[1]]
            
            trial_errors = [] 
            for i in [-1, 0, 1]:
                for j in [-1, 0, 1]:
                    for k in [-1, 0, 1]:
                        distance = np.linalg.norm(p2 - p1 + i*lattice_vectors[0] + j*lattice_vectors[1] + k*lattice_vectors[2])
                        trial_errors.append(np.power(distance - weight, 2))
            errors += np.min(trial_errors)
        print(errors)
        return errors

In [9]:
solution = minimize(objective, initial_positions, args=(edges, weights),
                    method='Powell')

# Check convergence status
if solution.success:
    print("Converged to a solution.")
else:
    print("Failed to converge:", solution.message)

1710.3452052441717
1710.3452052441717
1584.791707609805
1455.6237981355077
1392.4877829991594
1390.734285536984
1363.2463575217512
1367.103167849224
1365.339606076763
1363.4214398365377
1363.7728391813082
1363.317130535044
1363.2463575217512
1534.7161459912
1033.943795396127
779.4509202705036
815.261886164604
809.4214745367015
781.441115050244
778.5079132471831
778.6714586870285
778.587583735714
778.5079132471831
772.5927218079968
853.0363371348337
787.0736998857952
767.6379254362816
772.6830718724556
767.5694265833222
767.314899024344
767.7287296647434
767.2535714309174
767.2393138778012
767.2402002572743
767.2393138778012
804.0791835277311
745.9005091200437
744.9620838846581
747.3368594490995
744.5054928323356
744.5571980452852
744.7450660735448
744.4770078385751
744.485510416192
744.4770078385751
882.3888056165353
681.7936150288175
676.8406643146373
688.5294880347194
678.7444077489046
676.8987561190423
676.8393266135557
676.8625461620288
676.8393266135557
733.4187796006045
656.74207

KeyboardInterrupt: 

In [27]:
solution_attempt = solution.x.reshape(-1, 3)  # Reshape to 2D array

## Molecules

In [ ]:
if is_molecule:
    # Get the position of each atom in direct coordinates
    #direct_positions = graph_to_cartesian_positions(graph)
    #cartesian_positions = solution.x.reshape(-1, 3)*mw
    #cartesian_positions = solution.x.reshape(-1, 3)
    
    lattice_vectors     = np.array([[10, 0,   0],
                                    [0,   10, 0],
                                    [0,   0,   10]])
    cartesian_positions = solution_attempt

## Crystals

In [28]:
if not is_molecule:
    # Get the position of each atom in direct coordinates
    #direct_positions = graph_to_cartesian_positions(graph)
    #cartesian_positions = solution.x.reshape(-1, 3)*mw
    
    lattice_vectors     = solution_attempt[:3]
    cartesian_positions = solution_attempt[3:]

In [31]:
POSCAR_name = None

# Get name for the first line of the POSCAR
POSCAR_name = POSCAR_name or 'POSCAR from GenerativeModels'

# Clone the input graph to preserve the original structure
new_graph = temp.clone()

# Load and detach embeddings for the graph nodes
data_embeddings = new_graph.x.detach().cpu().numpy()

# Loading dictionary of available embeddings for atoms
available_embeddings = {}
with open('../VASP/atomic_masses.dat', 'r') as atomic_masses_file:
    for line in atomic_masses_file:
        key, mass, charge, electronegativity, ionization_energy = line.split()

        # Check if all information is present
        if all(val != 'None' for val in (mass, charge, electronegativity, ionization_energy)):
            available_embeddings[key] = np.array([mass, charge, electronegativity, ionization_energy], dtype=float)

# Get most similar atoms for each graph node and create a list of keys
keys = [find_closest_key(available_embeddings, emb) for emb in data_embeddings]

# Get elements' composition, concentration, and positions
POSCAR_composition, POSCAR_concentration, POSCAR_positions = composition_concentration_from_keys(keys, cartesian_positions)

In [32]:
# Write file
with open('CONTCAR', 'w') as POSCAR_file:
    # Delete previous data in the file
    POSCAR_file.truncate()
    
    # Write POSCAR's name
    POSCAR_file.write(f'{POSCAR_name}\n')

    # Write scaling factor (assumed to be 1.0)
    POSCAR_file.write('1.0\n')

    # Write lattice parameters (assumed to be orthogonal)
    np.savetxt(POSCAR_file, lattice_vectors, delimiter=' ')

    # Write composition (each different species, previously sorted)
    np.savetxt(POSCAR_file, [POSCAR_composition], fmt='%s', delimiter=' ')

    # Write concentration (number of each of the previous elements)
    np.savetxt(POSCAR_file, [POSCAR_concentration], fmt='%d', delimiter=' ')

    # Write position in cartesian form
    POSCAR_file.write('Cartesian\n')
    np.savetxt(POSCAR_file, POSCAR_positions, delimiter=' ')